In [1]:
# 이전의 gan과 달라진점 중간에 barch nomarize사용 

In [1]:
import torch
import torch.nn as nn
from torch.nn.modules import loss
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import os
import numpy as np
import time
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
class Discriminator(nn.Module):
    def __init__(self, channel_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # N x features_d x 32 x 32
            #input output 
        nn.Conv2d(channel_img,features_d,kernel_size=4,stride=2,padding=1),
        nn.LeakyReLU(0.2),
        self._block(features_d, features_d * 2, 4, 2, 1),
        self._block(features_d*2 , features_d * 4, 4, 2, 1),
        self._block(features_d*4 , features_d * 8, 4, 2, 1),
            
        nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0), 
          nn.Sigmoid(),   
        )
        
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            stride,
            padding,
            bias = False,
        ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
    def forward(self, x):
        return self.disc(x)

In [3]:
# channels_img : 64 *64
# features_d : 첫 layer에서 나오는 features의 dim
# output 마지막은 sigmoid를 사용해야하기 때문에 
# n * features_d *8*1*1d의 크기로 만든다 

In [4]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
        self._block(channels_noise, features_g *16,4,1,0),
        self._block(features_g * 16, features_g * 8, 4, 2, 1), # img: 8 x 8
        self._block(features_g * 8, features_g * 4, 4, 2, 1), # img: 16 x 16
        self._block(features_g * 4, features_g * 2, 4, 2, 1), # img: 32 x 32    
        
         nn.ConvTranspose2d(
          features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
        ),
        # Output: N x channels_img x 64 x 64
        nn.Tanh(),
      )
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False,
        ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def forward(self, x):
        return self.net(x)

In [5]:
# nn.ConvTranspose2d 형상을 늘려줌 업스케일링 과정 
# output N *channels_img * 64 *64

In [6]:
def initialize_weights(model):
  # Initializes weights according to the DCGAN paper
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, 0.0, 0.02) # mean 0, std 0.02

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4 # could also use two lrs, one for gen and one for disc (0.001)
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
NOISE_DIM = 50
NUM_EPOCHS = 10
FEATURES_DISC = 64
FEATURES_GEN = 64

In [8]:
transforms = transforms.Compose(
  [
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
    ),
  ]
)


In [16]:
dataset = datasets.MNIST(root="./Desktop", train=True, transform=transforms,
                       download=True)

In [10]:
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

# in the paper, it is stable when B1 is 0.5 than 0.9
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))  
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
writer_real = SummaryWriter(f"./runs/DCGAN_MNIST/DC_real")
writer_fake = SummaryWriter(f"./runs/DCGAN_MNIST/DC_fake")
step = 0

# BN, Dropout Train mode
gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(2, 2), stride=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

In [15]:
for epoch in range(NUM_EPOCHS):
  # Target labels not needed! ~~ unsupervised
  for batch_idx, (real, _) in enumerate(dataloader):
    real = real.to(device)
    noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
    fake = gen(noise)

    
    ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
    disc_real = disc(real).reshape(-1)
    lossD_real = criterion(disc_real, torch.ones_like(disc_real))
  
    disc_fake = disc(fake.detach()).reshape(-1)
    lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake) / 2
    disc.zero_grad()
    lossD.backward()
    opt_disc.step()

    ### Train Generator: min log(1 - D(G(z)) <-> max log(D(G(z))

    output = disc(fake).reshape(-1)
    lossG = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

    # Print losses occasionally and print to tensorboard
    if batch_idx % 100 == 0:
      print(
        f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
          Loss D : {lossD:.4f}, loss G: {lossG:.4f}"
      )

      with torch.no_grad():
              fake = gen(fixed_noise)
              # take out (up to) 32 examples
              img_grid_real = torchvision.utils.make_grid(
                  real[:32], normalize=True
              )
              img_grid_fake = torchvision.utils.make_grid(
                  fake[:32], normalize=True
              )

              writer_real.add_image("Real", img_grid_real, global_step=step)
              writer_fake.add_image("Fake", img_grid_fake, global_step=step)

      step += 1


Epoch [0/10] Batch 0/469           Loss D : 0.6952, loss G: 0.6983
Epoch [0/10] Batch 100/469           Loss D : 0.2223, loss G: 1.4774
Epoch [0/10] Batch 200/469           Loss D : 0.0532, loss G: 2.9184
Epoch [0/10] Batch 300/469           Loss D : 0.0335, loss G: 3.5027
Epoch [0/10] Batch 400/469           Loss D : 0.7622, loss G: 1.5555
Epoch [1/10] Batch 0/469           Loss D : 0.6399, loss G: 0.8227
Epoch [1/10] Batch 100/469           Loss D : 0.6521, loss G: 0.8574
Epoch [1/10] Batch 200/469           Loss D : 0.6677, loss G: 0.7123
Epoch [1/10] Batch 300/469           Loss D : 0.6845, loss G: 0.7408
Epoch [1/10] Batch 400/469           Loss D : 0.6594, loss G: 0.7385
Epoch [2/10] Batch 0/469           Loss D : 0.7031, loss G: 1.0660
Epoch [2/10] Batch 100/469           Loss D : 0.6619, loss G: 0.7637
Epoch [2/10] Batch 200/469           Loss D : 0.6678, loss G: 0.6997
Epoch [2/10] Batch 300/469           Loss D : 0.6550, loss G: 0.8313
Epoch [2/10] Batch 400/469           Los

In [18]:
for epoch in range(NUM_EPOCHS):
  # Target labels not needed! ~~ unsupervised
  for batch_idx, (real, _) in enumerate(dataloader):
    real = real.to(device)
    noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
    fake = gen(noise)
    print(real.shape)
    break

torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
torch.Size([128, 1, 64, 64])
